In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## First we need to login so the site thinks we're a normal user in a browser

In [2]:
login_url = 'https://www.agriconomie.com/login'
login_check_url = 'https://www.agriconomie.com/login_check'

client = requests.session()
client.headers.update({'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"})


# Retrieve the CSRF token first
soup = BeautifulSoup(client.get(login_url).content)
csrftoken = soup.find('input', dict(name='_csrf_token'))['value']

print(csrftoken)

login_data = {'_username': 'slava_tyvonchuk@yahoo.com', '_password': 'test_agriconomie2019', 
              '_csrf_token': csrftoken}
r = client.post(login_check_url, data=login_data, headers=dict(Referer=login_url))
r.status_code

0IkSRLdPY2DkeRIvsVrEO40NlbhP5LHaIAYpPawdRsM


200

In [3]:
dashboard_url = 'https://www.agriconomie.com/account/buyer/dashboard'
dashboard = client.get(dashboard_url)
print(dashboard.status_code)
dashboard_soup = BeautifulSoup(dashboard.content)

200


### If we see "Bonjour name" here then everything worked (we are logged in)

In [4]:
dashboard_soup.find('h1', {'class': 'title-large alternate'}).text

'Bonjour slava'

### Now we configure our post code (needed to see the price on this site)

In [5]:
post_code_data = {'town_email[town][_country]': 'FR', 'town_email[town][_postcode]': '27130', 
              'town_email[town][_select_town]': 9884, '_redirect': dashboard_url}
r = client.post('https://www.agriconomie.com/code-postal', data=login_data, headers=dict(Referer=dashboard_url))
r.status_code

200

### Just to check that it worked

In [6]:
pc_soup = BeautifulSoup(r.content)
pc_soup.find(lambda tag:tag.name=="span" and "Votre code" in tag.text).text

'\nVotre code postal actuel est le : 60000 - Allonne\n'

In [7]:
from pathlib import Path
Path('post_code.html').write_bytes(r.content)

254930

[Post code result](post_code.html)

[click](http://google.com)


### Now we write a function that can grab the product details for a given product URL

We want the product name and the different offers. Each offer is a tuple (price, packaging, treatment)

In [33]:
#2nd step : information from one link and its treatment?

url_detail = 'https://www.agriconomie.com/mais-grain-dkc5065-dekalb-410-460/p349971#_campaign=cat_listing_produit'

def grab_detail(url_detail):
    r_detail = client.get(url_detail)
    assert r_detail.status_code == 200
    detail_soup = BeautifulSoup(r_detail.content)
    product_name = detail_soup.find('h1', {'class': 'product-title'}).text.strip()
    offers_prices = [o.text for o in detail_soup.select('form.offer span.price span.price')]
    offers_packaging_treatment = [o.text for o in detail_soup.select('form.offer div.col-packaging p.cart-desc')]
    offers = []
    for ix, offer_price in enumerate(offers_prices):
        offer_packaging = offers_packaging_treatment[ix*2]
        offer_treatment = offers_packaging_treatment[ix*2+1]
        offer = (offer_price, offer_packaging, offer_treatment)
        offers.append(offer)
    return (product_name, offers)

grab_detail(url_detail)


('Maïs grain DKC 5065 demi-tardif DEKALB',
 [('141€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Force'),
  ('122€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo'),
  ('131€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Acceleron')])

In [34]:
#[o.text for o in detail_soup.select('form.offer span.price span.price')]
#[o.text for o in detail_soup.select('form.offer div.col-packaging p.cart-desc')]

## Now we get the list of all the products in the category we want "Semence de mais"

In [11]:
url_list = 'https://www.agriconomie.com/semences/semences-de-printemps/semences-de-mais/pc3409'
r = requests.get(url_list)
assert r.status_code == 200
r.content[:100]
soup = BeautifulSoup(r.content)

In [12]:
links = soup.select('.product-teaser')
#1st step : all links with details
test=[f'{i}' for i in links]
test=[i.split('<') for i in test]
test=[i[2].split('href=') for i in test]
test=[i[1].split('"') for i in test]
detail_urls=['https://www.agriconomie.com'+i[1] for i in test]
detail_urls

['https://www.agriconomie.com/mais-grain-dkc5065-dekalb-410-460/p349971',
 'https://www.agriconomie.com/mais-mixte-demi-prcoce-elzea-320/p363802',
 'https://www.agriconomie.com/mais-mixte-prcoce-mogana-270/p363798',
 'https://www.agriconomie.com/mais-fourrage-trs-prcoce-barcelos-230-240/p363797',
 'https://www.agriconomie.com/mas-fourrage-prcoce-lg-30260-limagrain-250-260/p363796',
 'https://www.agriconomie.com/mais-dkc-4598-demi-prcoce-dekalb/p363646',
 'https://www.agriconomie.com/mais-dkc-4391-demi-prcoce-dekalb/p363645',
 'https://www.agriconomie.com/mais-fourrager-dkc-3697-prcoce-dekalb/p363643',
 'https://www.agriconomie.com/mais-dkc-5182-dekalb/p363641',
 'https://www.agriconomie.com/mais-dkc-4569-demi-prcoce-dekalb/p363639',
 'https://www.agriconomie.com/mas-dkc-3872-demi-prcoce-dekalb/p363638',
 'https://www.agriconomie.com/mas-demi-prcoce-san-tadeo-rh-290-300/p363599',
 'https://www.agriconomie.com/mas-demi-prcoce-delrio-290/p363598',
 'https://www.agriconomie.com/mas-madi-pr

In [13]:
!pip install tqdm

In [17]:
from time import sleep
try:
    from tqdm import tqdm
except:
    tqdm = lambda x: x

products = []

for detail_url in tqdm(detail_urls):
    #print(detail_url)
    products.append(grab_detail(detail_url))
    sleep(2)  # Wait a few secs between visits so we don't raise alerts
    

100%|██████████████████████████████████████████| 48/48 [02:16<00:00,  2.79s/it]


In [18]:
products

[('Maïs grain DKC 5065 demi-tardif DEKALB',
  [('141€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Force'),
   ('122€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo'),
   ('131€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Acceleron')]),
 ('Mais mixte demi précoce - Elzea 320',
  [('60€00HT/ Dose', 'Dose 50 MG - Base', 'Traitement : Redigo')]),
 ('Mais mixte précoce - Mogana 270',
  [('60€00HT/ Dose', 'Dose 50 MG - Base', 'Traitement : Redigo')]),
 ('Mais mixte très précoce - Barcelos 230-240',
  [('60€00HT/ Dose', 'Dose 50 MG - Base', 'Traitement : Redigo')]),
 ('Maïs fourrage précoce LG 30.260 HDi Limagrain 250-260',
  [('91€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo'),
   ('120€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Force')]),
 ('Maïs grain DKC 4598 Demi-précoce DEKALB',
  [('140€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Force'),
   ('149€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redigo+Force+Acceleron'),
   ('121€00HT/ Dose', 'Dose 50 MG', 'Traitement : Redig

In [41]:
products_as_dict = []

max_offers = max([len(o) for n, o in products])
print(max_offers)

for p in products:
    name, offers = p
    
    dp = {'name': name}
    
    for ix, offer in enumerate(offers):
        price, packaging, treatment = offer
        price, unit = price.split("/")
        dp[f'offer_{ix+1}_price'] = price.strip()
        dp[f'offer_{ix+1}_unit'] = unit.strip()
        dp[f'offer_{ix+1}_packaging'] = packaging
        dp[f'offer_{ix+1}_treatment'] = treatment
    
    products_as_dict.append(dp)
products_as_dict

4


[{'name': 'Maïs grain DKC 5065 demi-tardif DEKALB',
  'offer_1_price': '141€00HT',
  'offer_1_unit': 'Dose',
  'offer_1_packaging': 'Dose 50 MG',
  'offer_1_treatment': 'Traitement : Redigo+Force',
  'offer_2_price': '122€00HT',
  'offer_2_unit': 'Dose',
  'offer_2_packaging': 'Dose 50 MG',
  'offer_2_treatment': 'Traitement : Redigo',
  'offer_3_price': '131€00HT',
  'offer_3_unit': 'Dose',
  'offer_3_packaging': 'Dose 50 MG',
  'offer_3_treatment': 'Traitement : Redigo+Acceleron'},
 {'name': 'Mais mixte demi précoce - Elzea 320',
  'offer_1_price': '60€00HT',
  'offer_1_unit': 'Dose',
  'offer_1_packaging': 'Dose 50 MG - Base',
  'offer_1_treatment': 'Traitement : Redigo'},
 {'name': 'Mais mixte précoce - Mogana 270',
  'offer_1_price': '60€00HT',
  'offer_1_unit': 'Dose',
  'offer_1_packaging': 'Dose 50 MG - Base',
  'offer_1_treatment': 'Traitement : Redigo'},
 {'name': 'Mais mixte très précoce - Barcelos 230-240',
  'offer_1_price': '60€00HT',
  'offer_1_unit': 'Dose',
  'offer_1_

In [42]:
df_products = pd.DataFrame.from_records(products_as_dict)
df_products.head()

,name,offer_1_packaging,offer_1_price,offer_1_treatment,offer_1_unit,offer_2_packaging,offer_2_price,offer_2_treatment,offer_2_unit,offer_3_packaging,offer_3_price,offer_3_treatment,offer_3_unit,offer_4_packaging,offer_4_price,offer_4_treatment,offer_4_unit
0,Maïs grain DKC 5065 demi-tardif DEKALB,Dose 50 MG,141€00HT,Traitement : Redigo+Force,Dose,Dose 50 MG,122€00HT,Traitement : Redigo,Dose,Dose 50 MG,131€00HT,Traitement : Redigo+Acceleron,Dose,NaN,NaN,NaN,NaN
1,Mais mixte demi précoce - Elzea 320,Dose 50 MG - Base,60€00HT,Traitement : Redigo,Dose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mais mixte précoce - Mogana 270,Dose 50 MG - Base,60€00HT,Traitement : Redigo,Dose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mais mixte très précoce - Barcelos 230-240,Dose 50 MG - Base,60€00HT,Traitement : Redigo,Dose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Maïs fourrage précoce LG 30.260 HDi Limagrain ...,Dose 50 MG,91€00HT,Traitement : Redigo,Dose,Dose 50 MG,120€00HT,Traitement : Redigo+Force,Dose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_products.to_pickle('webscrap_data_mais.pkl')